In [1]:
import pandas as pd 
from sklearn.metrics import classification_report

from utils import predictions

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## Проверка на тестовой выборке 

В процессе построения модели выделялась тестовая выборка, размер которой - 10% от всей базы данных. Ни одна из моделей ни разу не видела эти данные и не взаимодействовала с ними, поэтому проверка на тестовой выборке - чистый эксперимент. Загрузим тестовые данные и построим предсказания, чтобы оценить эффективность моделей. 

In [2]:
test_x = pd.read_csv('data/test_x.csv')
test_y = pd.read_csv('data/test_y.csv')['comment_kind']

In [3]:
forest_pred = predictions.forest_pipe.predict(test_x)
svc_pred = predictions.svc_pipe.predict(test_x)
voting_pred = predictions.voting_pipe.predict(test_x)

In [4]:
print(classification_report(test_y, forest_pred, digits=4))

              precision    recall  f1-score   support

           0     0.6733    0.3886    0.4928       175
           1     0.7794    0.9197    0.8438       411

    accuracy                         0.7611       586
   macro avg     0.7263    0.6541    0.6683       586
weighted avg     0.7477    0.7611    0.7389       586



Точность (precision) модели случайного леса на валидационной выборке достигала 73.88 %, на тестовой она составляет немногим меньше - 72.63%. Это всё ещё высокий показатель, который превышает вероятность угадать методом рандомного тыка. 

In [5]:
print(classification_report(test_y, svc_pred, digits=4))

              precision    recall  f1-score   support

           0     0.6902    0.7257    0.7075       175
           1     0.8806    0.8613    0.8708       411

    accuracy                         0.8208       586
   macro avg     0.7854    0.7935    0.7892       586
weighted avg     0.8237    0.8208    0.8221       586



Точность (precision) модели случайного леса на валидационной выборке достигала 78.98 %, на тестовой она практически та же, отличаются доли процента. Это значит, что модель действительно показывает высокие результаты на совершенно неизвестных ей текстах. 

In [6]:
print(classification_report(test_y, voting_pred, digits=4))

              precision    recall  f1-score   support

           0     0.6902    0.7257    0.7075       175
           1     0.8806    0.8613    0.8708       411

    accuracy                         0.8208       586
   macro avg     0.7854    0.7935    0.7892       586
weighted avg     0.8237    0.8208    0.8221       586



Точность (precision) объединённой модели на валидационной выборке достигала 79.36 %, на тестовой она в точности совпадает с показателями ядерного метода опорных векторов (видимо, его значения на тестовой выборке оказывают чуть большее воздействие на усредняемые вероятности, чем результаты модели случайного леса). Это всё ещё высокий показатель, который превышает вероятность угадать методом рандомного тыка. 

## Применение к собственному тексту

Напишем два текста, которые могли бы быть не самыми качественными отзывами на фильмы, и попробуем определить их вид с помощью каждой из моделей. 

In [11]:
# Предполагается, что текст негативный 
text1 = 'Мультик совершенно не заслуживает внимания. Никогда бы не подумала, что такое могут вообще снимать. Это могло бы быть хорошим произведением искусства, но сценаристу явно забыли заплатить, а режиссёр как будто всё время отсутствовал на площадке. В общем, я бы не советовала это смотреть никому, кроме любителей плохих мультиков и плохих режиссёров (чтобы потешить своё самолюбие).'

# Предполагается, что текст позитивный 
text2 = 'В конце я аж расплакалась от того, насколько это правдоподобная история о любви. Робот похож на человека больше, чем сами люди похожи на людей. Интересная подача истории о том, каким могло бы быть человеческое будущее, если бы его захватили технологии или если бы человечество само дало технологиям полностью себя поработить. Забавно, что роботы не целуются, а обмениваются импульсами, - метафора что надо.'

# Предполагается, что текст нейтрально-негативный
text3 = 'Мультфильм ни о чём. В целом, можно потратить на него время, если других вариантов нет, но на премию он не претендует. Хорошая картинка, бессмысленный сюжет. Получилось достаточно сыро: явно есть потенциал у этой картины, но сценаристам и аниматорам ещё нужно совершенствоваться.'

In [8]:
text1_results = predictions.make_prediction(text1, model='all')

>> Random Forest Classifier
Вероятность негативного или нейтрального: 42.53 %
Вероятность позитивного: 57.47 %
>> Вердикт: ПОЗИТИВНЫЙ

>> SVC
Вероятность негативного или нейтрального: 59.22 %
Вероятность позитивного: 40.78 %
>> Вердикт: НЕГАТИВНЫЙ ИЛИ НЕЙТРАЛЬНЫЙ

>> Voting Classifier
Вероятность негативного или нейтрального: 59.22 %
Вероятность позитивного: 40.78 %
>> Вердикт: НЕГАТИВНЫЙ ИЛИ НЕЙТРАЛЬНЫЙ



Модель случайного леса почему-то распознала в негативном тексте что-то позитивное, и вероятность позитивного превысила (пусть и незначительно), в результате чего модель дала неверное предсказание. Ядерный метод опорных векторов и объединённая модель справились корректно. 

In [9]:
text2_results = predictions.make_prediction(text2, model='all')

>> Random Forest Classifier
Вероятность негативного или нейтрального: 29.62 %
Вероятность позитивного: 70.38 %
>> Вердикт: ПОЗИТИВНЫЙ

>> SVC
Вероятность негативного или нейтрального: 2.08 %
Вероятность позитивного: 97.92 %
>> Вердикт: ПОЗИТИВНЫЙ

>> Voting Classifier
Вероятность негативного или нейтрального: 2.08 %
Вероятность позитивного: 97.92 %
>> Вердикт: ПОЗИТИВНЫЙ



С позитивным текстом все модели справились успешно, однако модель случайного леса сомневается больше других. 

In [12]:
text3_results = predictions.make_prediction(text3, model='all')

>> Random Forest Classifier
Вероятность негативного или нейтрального: 36.34 %
Вероятность позитивного: 63.66 %
>> Вердикт: ПОЗИТИВНЫЙ

>> SVC
Вероятность негативного или нейтрального: 71.41 %
Вероятность позитивного: 28.59 %
>> Вердикт: НЕГАТИВНЫЙ ИЛИ НЕЙТРАЛЬНЫЙ

>> Voting Classifier
Вероятность негативного или нейтрального: 71.41 %
Вероятность позитивного: 28.59 %
>> Вердикт: НЕГАТИВНЫЙ ИЛИ НЕЙТРАЛЬНЫЙ



Случайный лес снова нашёл больше позитива (оптимистичная модель получилась), остальные две модели справились весьма удачно. 